This restaurant simulation is borrowing heavily from BANK_RENEGE in the simpy documentation examples. [Link](https://simpy.readthedocs.io/en/latest/examples/bank_renege.html)

In [12]:
import simpy
env = simpy.Environment()
#a restaurant is a resource with some capacity  
#for now, it's a rather small restaurant
restaurant = simpy.Resource(env, capacity = 4)

# A customer shows up at a restaurant and waits to be seated, is served, and leaves
def customer(env, name):
    while True: 
        enter = env.now
        wait_time = 5
        print("Here's customer %s waiting" % name) 
        yield env.timeout(wait_time)

        print('Customer %s gets seated at %d' % (name, env.now))
        eat_time = 30
        yield env.timeout(eat_time)
        print ("and now customer %s will pay and leave at %d" % (name, env.now))
        
# but now we need to instantiate some customers 
env.process(customer(env, 1))
env.run(until=100)


Here's customer 1 waiting
Customer 1 gets seated at 5
and now customer 1 will pay and leave at 35
Here's customer 1 waiting
Customer 1 gets seated at 40
and now customer 1 will pay and leave at 70
Here's customer 1 waiting
Customer 1 gets seated at 75


In [ ]:
# now there will be more customers 

In [13]:
env = simpy.Environment()
for i in range(4): 
    env.process(customer(env, i))
   
env.run(until=100) 

Here's customer 0 waiting
Here's customer 1 waiting
Here's customer 2 waiting
Here's customer 3 waiting
Customer 0 gets seated at 5
Customer 1 gets seated at 5
Customer 2 gets seated at 5
Customer 3 gets seated at 5
and now customer 0 will pay and leave at 35
Here's customer 0 waiting
and now customer 1 will pay and leave at 35
Here's customer 1 waiting
and now customer 2 will pay and leave at 35
Here's customer 2 waiting
and now customer 3 will pay and leave at 35
Here's customer 3 waiting
Customer 0 gets seated at 40
Customer 1 gets seated at 40
Customer 2 gets seated at 40
Customer 3 gets seated at 40
and now customer 0 will pay and leave at 70
Here's customer 0 waiting
and now customer 1 will pay and leave at 70
Here's customer 1 waiting
and now customer 2 will pay and leave at 70
Here's customer 2 waiting
and now customer 3 will pay and leave at 70
Here's customer 3 waiting
Customer 0 gets seated at 75
Customer 1 gets seated at 75
Customer 2 gets seated at 75
Customer 3 gets seate

Not bad, but I wanted a gap between customers, which means I need to model arrival of customers.  Borrowing the "source" pattern in BANK_RENEGE example.  

In [14]:
# RANDOM_SEED = 42 leaving out randomness for now
NEW_CUSTOMERS = 10  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds

def source(env, number, interval, restaurant):
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, restaurant)
        env.process(c)
        yield env.timeout(interval)


def customer(env, name, restaurant):
    enter = env.now
    wait_time = 0
    print("Here's customer %s waiting at %d" % (name, env.now) )
    yield env.timeout(wait_time)

    with restaurant.request() as req:
        yield req
        
        print('Customer %s gets seated at %d' % (name, env.now))
        eat_time = 60
        yield env.timeout(eat_time)
        
        print ("and now customer %s will pay and leave at %d" % (name, env.now))
        # this is when the restaurant gets paid 
    
    
env=simpy.Environment()
restaurant = simpy.Resource(env, capacity = 4)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, restaurant))
env.run(until=350)

Here's customer Customer00 waiting at 0
Customer Customer00 gets seated at 0
Here's customer Customer01 waiting at 10
Customer Customer01 gets seated at 10
Here's customer Customer02 waiting at 20
Customer Customer02 gets seated at 20
Here's customer Customer03 waiting at 30
Customer Customer03 gets seated at 30
Here's customer Customer04 waiting at 40
Here's customer Customer05 waiting at 50
and now customer Customer00 will pay and leave at 60
Here's customer Customer06 waiting at 60
Customer Customer04 gets seated at 60
and now customer Customer01 will pay and leave at 70
Here's customer Customer07 waiting at 70
Customer Customer05 gets seated at 70
and now customer Customer02 will pay and leave at 80
Here's customer Customer08 waiting at 80
Customer Customer06 gets seated at 80
and now customer Customer03 will pay and leave at 90
Here's customer Customer09 waiting at 90
Customer Customer07 gets seated at 90
and now customer Customer04 will pay and leave at 120
Customer Customer08 ge

Coolio.  TIL that capacity is respected even if I don't specifically invoke any behavior about it. (Today, on more careful inspection, I don't think that's true.  The restaurant is getting ignored.)

Now I want to make it possible for the restaurant to collect revenue.  I think the right model here is a class, Restaurant, which has a resource (those scarce tables)and a revenue associated with it.  

In [31]:
class Restaurant(simpy.Resource):
    
    
    # temporary simplification, party size = table size = 1
    
    def __init__(self, env, numseats, revenue=0):
        simpy.Resource.__init__(self, env, capacity=numseats)
        self.revenue = revenue
        
    def pay(self, check): 
        self.revenue = check + self.revenue
        print ('at %d revenue is %d' % (env.now, self.revenue)) 
        


    
        
    

In [42]:
import random 

# RANDOM_SEED = 42 leaving out randomness for now
NEW_CUSTOMERS = 10  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1
MAX_PATIENCE = 20 

def source(env, number, interval, restaurant):
    for i in range(number):
        # generate a patience threshold for this customer 
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE) 
        c = customer(env, 'Customer%02d' % i, restaurant, patience)
        env.process(c)
        yield env.timeout(interval)


def customer(env, name, restaurant, patience = MIN_PATIENCE ):
    enter = env.now
    wait_time = 0
    print("Here's customer %s waiting at %d" % (name, env.now) )
    yield env.timeout(wait_time)
    
    # An impatient customer will only wait for 1 tick  - right now all customers
    # are impatient. 

        
    with restaurant.request() as req:
        
        queue = yield req | env.timeout(patience)
        
        if req in queue: 
            print('Customer %s gets seated at %d' % (name, env.now))
            eat_time = 60
            yield env.timeout(eat_time)
        
            print ("and now customer %s will pay and leave at %d" % (name, env.now))
            # this is when the restaurant gets paid - every check is $10 for now
            restaurant.pay(10) 
        else:
            print ("Customer %s abandoned at time %d" % (name, env.now))
                   

In [43]:
env=simpy.Environment()
myrestaurant = Restaurant(env, numseats = 4)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, myrestaurant))
env.run(until=200)

Here's customer Customer00 waiting at 0
Customer Customer00 gets seated at 0
Here's customer Customer01 waiting at 10
Customer Customer01 gets seated at 10
Here's customer Customer02 waiting at 20
Customer Customer02 gets seated at 20
Here's customer Customer03 waiting at 30
Customer Customer03 gets seated at 30
Here's customer Customer04 waiting at 40
Here's customer Customer05 waiting at 50
Customer Customer04 abandoned at time 53
and now customer Customer00 will pay and leave at 60
at 60 revenue is 10
Here's customer Customer06 waiting at 60
Customer Customer05 gets seated at 60
and now customer Customer01 will pay and leave at 70
at 70 revenue is 20
Here's customer Customer07 waiting at 70
Customer Customer06 gets seated at 70
and now customer Customer02 will pay and leave at 80
at 80 revenue is 30
Here's customer Customer08 waiting at 80
Customer Customer07 gets seated at 80
Customer Customer08 abandoned at time 81
and now customer Customer03 will pay and leave at 90
at 90 revenue